In [18]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
tf.config.run_functions_eagerly(True)


print(os.getcwd())

/Users/sumitsingh/Documents/GitHub/mlproject/StockAppFrontend/ml_model


In [11]:
data = pd.read_csv(
    "/Users/sumitsingh/Documents/GitHub/mlproject/StockAppFrontend/ml_model/source/indexProcessed.csv"
)
prices = data["Close"].values

scaler = MinMaxScaler() #normalising data into fixed min-max val
prices_scaled = scaler.fit_transform(prices.reshape(-1, 1))

print(data.head())

  Index        Date         Open         High          Low        Close  \
0   HSI  1986-12-31  2568.300049  2568.300049  2568.300049  2568.300049   
1   HSI  1987-01-02  2540.100098  2540.100098  2540.100098  2540.100098   
2   HSI  1987-01-05  2552.399902  2552.399902  2552.399902  2552.399902   
3   HSI  1987-01-06  2583.899902  2583.899902  2583.899902  2583.899902   
4   HSI  1987-01-07  2607.100098  2607.100098  2607.100098  2607.100098   

     Adj Close  Volume    CloseUSD  
0  2568.300049     0.0  333.879006  
1  2540.100098     0.0  330.213013  
2  2552.399902     0.0  331.811987  
3  2583.899902     0.0  335.906987  
4  2607.100098     0.0  338.923013  


In [4]:
# Prepare Features
window_size = 60  # Number of past days to use as input
X, y = [], []
for i in range(window_size, len(prices_scaled)):
    X.append(prices_scaled[i - window_size : i, 0])
    y.append(prices_scaled[i, 0])

X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape for LSTM

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [26]:
# Build LSTM Model
model = tf.keras.Sequential(
    [
        tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
        tf.keras.layers.LSTM(50, return_sequences=False),
        tf.keras.layers.Dense(25),
        tf.keras.layers.Dense(1),
    ]
)

model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Save the Model
model.save("stock_prediction_model.h5")

# Predict
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

Epoch 1/10


/Users/sumitsingh/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2605/2605 ━━━━━━━━━━━━━━━━━━━━ 444s 171ms/step - loss: 3.1663e-04 - val_loss: 1.7224e-05
Epoch 2/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 5705s 2s/step - loss: 3.6003e-05 - val_loss: 1.3120e-05
Epoch 3/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 455s 175ms/step - loss: 2.1587e-05 - val_loss: 2.2466e-05
Epoch 4/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 444s 171ms/step - loss: 2.7482e-05 - val_loss: 1.4542e-05
Epoch 5/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 438s 168ms/step - loss: 1.1717e-05 - val_loss: 2.8681e-05
Epoch 6/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 437s 168ms/step - loss: 2.2579e-05 - val_loss: 1.0251e-05
Epoch 7/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 436s 167ms/step - loss: 1.3786e-05 - val_loss: 9.4468e-06
Epoch 8/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 23738s 9s/step - loss: 1.5239e-05 - val_loss: 9.5588e-06
Epoch 9/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 18609s 7s/step - loss: 1.8275e-05 - val_loss: 1.5372e-05
Epoch 10/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 782s 300ms/step - loss: 2.2875e-05 - val_loss: 1.1077e-05


652/652 ━━━━━━━━━━━━━━━━━━━━ 80s 123ms/step


In [27]:
from tensorflow.keras.models import load_model

model = load_model("stock_prediction_model.h5")

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,903 (124.62 KB)

 Trainable params: 31,901 (124.61 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [9]:
model_json = model.to_json()

with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

In [28]:
# Install the required package
%pip install tensorflow

from tensorflow.keras.utils import plot_model

plot_model(
    model, to_file="model_architecture.png", show_shapes=True, show_layer_names=True
)

Note: you may need to restart the kernel to use updated packages.
You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [ ]:
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
model.evaluate(X_test, y_test)

Epoch 1/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 838s 322ms/step - loss: 2.4668e-05 - val_loss: 1.0236e-05
Epoch 2/10
2605/2605 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - loss: 1.6638e-05